In [1]:
import * as duckdb from 'https://cdn.jsdelivr.net/npm/@duckdb/duckdb-wasm@1.29.0/+esm';

const JSDELIVR_BUNDLES = duckdb.getJsDelivrBundles();

// Select a bundle based on browser checks
const bundle = await duckdb.selectBundle(JSDELIVR_BUNDLES);

const worker_url = URL.createObjectURL(
  new Blob([`importScripts("${bundle.mainWorker}");`], { type: 'text/javascript' })
);


const worker = new Worker(worker_url);
const logger = new duckdb.ConsoleLogger();
const db = new duckdb.AsyncDuckDB(logger, worker);
await db.instantiate(bundle.mainModule, bundle.pthreadWorker);
URL.revokeObjectURL(worker_url);

In [2]:
conn = await db.connect()

In [3]:
const result = await conn.query("SELECT 42 as col")

pprint(result.toArray().map(row => row.toJSON()))

[object Object]
[
  {
    "col": 42
  }
]


In [8]:
const url = 'https://raw.githubusercontent.com/plotly/datasets/master/2015_flights.parquet';

const response = await fetch(url);
if (!response.ok){
    throw Error("Failed to fetch")
}

const buffer = await response.arrayBuffer();
const uint8Array = new Uint8Array(buffer);
await db.registerFileBuffer('2015_flights.parquet', uint8Array);


const data = await conn.query(`
SELECT * FROM read_parquet('2015_flights.parquet') LIMIT 1;
`
)

const rows = data.toArray().map(row => {
  const jsonRow = row.toJSON();
  for (const key in jsonRow) {
    if (typeof jsonRow[key] === 'bigint') {
      jsonRow[key] = jsonRow[key].toString();
    }
  }
  return jsonRow;
});

pprint(rows);

[object Object]
[
  {
    "DEPARTURE_DELAY": -11,
    "ARRIVAL_DELAY": -22,
    "DISTANCE": "1448",
    "SCHEDULED_DEPARTURE": 0.08333333333333333
  }
]
